In [51]:
# /*==========================================================================================*\
# **                        _           _ _   _     _  _         _                            **
# **                       | |__  _   _/ | |_| |__ | || |  _ __ | |__                         **
# **                       | '_ \| | | | | __| '_ \| || |_| '_ \| '_ \                        **
# **                       | |_) | |_| | | |_| | | |__   _| | | | | | |                       **
# **                       |_.__/ \__,_|_|\__|_| |_|  |_| |_| |_|_| |_|                       **
# \*==========================================================================================*/


# Author: Bùi Tiến Thành (@bu1th4nh)
# Date: 2022/12/26 14:13 
# CTTN Toán tin K64

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import os
import re
import ujson as json
import os
import ot
from dtaidistance import dtw

# Figure size
height = 20
width  = 8

## Input and Normalization

In [52]:
Ariel = pd.read_parquet("../DataWater_train_cleansed_phase3.parquet")

In [53]:
attributes  = Ariel.columns
mean        = Ariel.describe().loc["mean", :].to_numpy()
std         = Ariel.describe().loc["std", :].to_numpy()
D           = len(attributes)
T           = len(Ariel)

print("Mean: ", mean);
print("Std:  ", std);

Mean:  [7.37695093e+00 2.48333939e+04 5.89920648e+00 1.05507809e+02
 3.60180180e+00 7.34455756e+01 1.86958681e+01 3.66098082e+02
 2.75782771e+01 2.98926587e+01]
Std:   [1.86057021e+00 1.82981284e+04 2.06528890e+00 1.09194459e+02
 5.26669067e-01 1.77687466e+01 2.72835960e+00 2.66349959e+02
 3.85663553e+00 1.10595931e+00]


In [54]:
# Do data được đo theo giờ nên ta có thể chuyển qua thời gian tương đối
initial_time = Ariel.index[0];
Ariel.reset_index(drop=True, inplace=True)
for i, col in enumerate(Ariel.columns):
    Ariel[col] = Ariel[col].apply(lambda x: (x - mean[i]) / std[i])

## Xây dựng ma trận đánh dấu

In [55]:
mask = ~pd.isnull(Ariel)
display(mask.head(5))

,pH,EC,DO,TSS,TN,TP,TOC,ORP,Temp,TEMP
0,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True


## Xây dựng ma trận delta dựa theo số bước


In [56]:
n_steps = 96

def build_delta(start):
    delta = np.zeros((n_steps, D))
    for i in range(1, n_steps):
        delta[i] = np.ones(D) + (~mask.loc[start + i - 1, :].astype('int')) * delta[i - 1]
    return delta


## Xây dựng mẫu

In [57]:
idx_list = []
start = 0;
end = 0;

for i in range(T - n_steps + 1):
    if(i % 12 == 0):
        idx_list.append((i, i + n_steps))
if(idx_list[-1][1] != T):
    idx_list.append((T - n_steps, T))


print(len(idx_list))

2350


## Xây dựng JSON

In [58]:
def parse_rec(values, masks, deltas):

    # only used in GRU-D
    forwards = pd.DataFrame(values).fillna(method='ffill').fillna(0.0).iloc[:,:].values

    rec = {}

    rec['values'] = np.nan_to_num(values).tolist()
    rec['masks'] = masks.astype('int32').tolist()
    rec['forwards'] = forwards.tolist()
    rec['deltas'] = deltas.tolist()

    return rec



save_path = 'data/json/split'
if not os.path.exists(save_path):
    os.makedirs(save_path)
for i, (start, end) in tqdm(enumerate(idx_list), desc='Building JSON'):
    values = Ariel.iloc[start:end].to_numpy()
    delta  = build_delta(start)
    mask_   = mask.iloc[start:end].to_numpy()
    label  = Ariel.iloc[end].copy().fillna(0.0).to_numpy()

    rec = {'label': label.tolist()}
    rec['forward']  = parse_rec(values, mask_, delta)
    rec['backward'] = parse_rec(values, mask_, delta)
    rec = json.dumps(rec)

    save_file = os.path.join(save_path, str(i))
    with open(save_file,'w') as f:
        f.write(rec)

Building JSON: 2349it [02:41, 14.51it/s]


IndexError: single positional indexer is out-of-bounds